In [117]:
import numpy as np

# Use this function when you have a pcS model collection, and which to classify a new observation (i.e. label it's cluster by index)
def classifyOb(x, modelCollection):
	#x is a single 1-by-n observation. modelCollection is a list of pcS models in the form: (X,A,mu) -see paper
	#score observation
	scores = np.zeros(len(modelCollection))
	for i in range(0,len(scores)):
		#transform the observation to the new basis
		xtag = (x - modelCollection[i][2])
		transPoint = np.dot(xtag,modelCollection[i][1])
		scores[i] = np.dot(transPoint,transPoint.T)
	scores = np.sqrt(scores)
	
	#Find most likely model
	bestScore = min(scores)
	return np.where(scores==bestScore)[0][0]

def modelCluster(X, pVar):
	#perform pca on X
	X = np.array(X)
	mu = np.mean(X.T,axis=1)
	M = (X-mu).T # subtract the mean (along columns)
	[latent,coeff] = np.linalg.eig(np.cov(M)) # attention:not always sorted
	order = [i[0] for i in sorted(enumerate(np.absolute(latent)), key=lambda x:x[1],reverse=True)]
	latent = latent[order]
	coeff = coeff[:,order]	
	contributions = latent/sum(latent)
	
	#only retain the PCs that contain the Pvar percent of variance of the data.
	if pVar == 1:
		numPCs = len(latent)
	else:
		numPCs = next((i for i, v in enumerate(np.cumsum(contributions)) if v >= pVar),len(latent)-1)+1
	
	Ltag = latent[0:numPCs]
	D = np.sqrt(Ltag); 
	C = coeff[:,0:numPCs]
	A = np.dot(C,np.diag(D))
	return (X,A,mu)


def pcS(X, pcS_args): #dataset is an ordered rdd with each row being an observation 
  #pcS_args: (DriftThreshold, DriftSize -must be greater than n, PercentVareince, ModelMemorySize -greater then driftSize)
	if isinstance(X,list):
		X = np.array(X)
	n = X.shape[1] #the number of features
	m = X.shape[0]
	
	driftThr = pcS_args[0] #Drift Threshold -set to 1/4th the dataset's std
	driftSize = pcS_args[1] #Buffer size: max drift size -set to 10 times the number of features
	pVar = pcS_args[2] #Percent variance to retain
	modelMemSize = pcS_args[3] #the memory size of each model - 10times the drift size
	
	modelCollection = list()
	modelCollection.append(modelCluster(X[0:driftSize,:],pVar)) #initial model
	
	labels = [0]*driftSize
	driftBuffer = list()
	curModel = 0
	
	#Begin reading stream
	print "beginning stream"
	for x in X[driftSize+1:,:]: #continue pcStream
		# process next observation
		#score observation
		scores = np.zeros(len(modelCollection))
		for i in range(0,len(scores)):
			#transform the observation to the new basis
			xtag = (x - modelCollection[i][2])
			transPoint = np.dot(xtag,modelCollection[i][1])
			scores[i] = np.dot(transPoint,transPoint.T)
		scores = np.sqrt(scores)
		#Find most likely model
		bestScore = min(scores)
		bestModl = np.where(scores==bestScore)[0][0]
		del scores
		
		#Detect drift
		if bestScore > driftThr:
			# Add the drifter to a buffer
			driftBuffer.append(x)
			
			#Check if the buffer is full (has enough to make a substantial model
			if len(driftBuffer) == driftSize:
				modelCollection.append(modelCluster(driftBuffer,pVar))
				curModel = len(modelCollection)-1
				labels = labels + [curModel]*len(driftBuffer)
				del driftBuffer[:]
				
		
		else: # there was no drift
			#Check for partial drift
			if len(driftBuffer) > 0:
				x = [x] + driftBuffer
				labels = labels + [bestModl]*len(driftBuffer)
				del driftBuffer[:]
			
			# assign current observation
			curModel = bestModl
			if isinstance(x,np.ndarray):
				x = [x]
			
			modelCollection[curModel] = modelCluster( (x+list(modelCollection[curModel][0]))[0:modelMemSize], pVar)
			labels.append(curModel)
			#time.sleep(1) #for debugging...   					
	# empty dirftBuffer one last time
	modelCollection[curModel] = modelCluster( (driftBuffer + list(modelCollection[curModel][0]))[0:modelMemSize], pVar)
	labels = labels + [curModel]*len(driftBuffer)
	
	# End of pcStream:
	return (modelCollection, labels)


def genDataset(conceptDurations,conceptScale,ConceptOffsetScale,numConcepts,numFeatures):
	conceptScale = float(conceptScale)
	ConceptOffsetScale = float(ConceptOffsetScale)		
	curConcept = 0
	while curConcept < numConcepts:
		curConcept += 1
		curObs = 0 # there are "conceptDurations" of observation in each concept
		while curObs < conceptDurations:
			curObs += 1
			yield (np.random.randn(1,numFeatures)*(np.random.rand(1,numFeatures)*conceptScale)-np.random.rand(1,numFeatures)*ConceptOffsetScale)[0]

In [148]:
import pandas as pd
import numpy as np
X_train=np.load('x_train_mawilab.npy')
X_test=np.load('x_test_mawilab.npy')
y_train=np.load('y_train_mawilab.npy')#pd.read_csv('../data/maiwilab300000.tsv',sep='\t')['label']
y_test=np.load('y_test_mawilab.npy')
x=np.concatenate((X_train,X_test),axis=0)
y=np.concatenate((y_train,y_test),axis=0)

In [149]:
x=pd.DataFrame(x)
cols_to_norm = [ i for i in range(0,63) ]

x.loc[:, cols_to_norm] = (x[cols_to_norm] - x[cols_to_norm].mean()) / x[cols_to_norm].std()
x=x.values

In [155]:

import timeit

# If you use the source code or impliment pcStream, please cite the following paper:
# Mirsky, Yisroel, Bracha Shapira, Lior Rokach, and Yuval Elovici. "pcStream: A Stream Clustering Algorithm for Dynamically Detecting and Managing Temporal Contexts." In Advances in Knowledge Discovery and Data Mining (PAKDD), pp. 119-133. Springer International Publishing, 2015.

#### Demo ####

# generate a test dataset
conceptDuration = 1000
conceptDistributionScale = 1
conceptOffset = 3
numSequentialConcepts = 10
n = 63 #number of features

# DataGen = genDataset(conceptDuration,conceptDistributionScale,conceptOffset,numSequentialConcepts,n) #pcStream.genDataset is a python generator for testing purposes. It generates random sequential overalping normal distributions
# Dataset = list()

# for x in DataGen:
#     Dataset.append(x)
Dataset=x[87500:]
m = len(Dataset) #length of stream

# Run pcStream
driftThreshold = 8.7 # note, it is generally reccoemnded to zscore the dataset before hand. In which case, the driftThreshold is typically slightly above 1
maxDriftSize = int(n*1.5)
percentVarience = 0.60
modelMemory = 10000

pcS_args = (driftThreshold,maxDriftSize,percentVarience,modelMemory) #pcStream argumetns are passed to the function as a tuple
start = timeit.default_timer()	
Result =pcS(Dataset,pcS_args) #Result is the tuple (pcStreamModelCollection, clusterLabels)
stop = timeit.default_timer()
modelCollection  = Result[0]

print "============  Done.  ============"
print "It took " + str(stop-start) + " seconds to train pcStream over a stream of " + str(m) + "  " + str(n) + "-dimensional observations."
print "   Number of clusters found: " + str(len(modelCollection))
print "   The clusters have the following number of assigned observations:  "
for cluster in range(0,len(modelCollection)): #a pcStream cluster has the form (X,A,mu) -see the paper
    print "       Cluster " + str(cluster) + ": " + str(len(modelCollection[cluster][0]))

print "  "
print "Predicting (scoring) new observation x:"
#x =genDataset(conceptDuration,conceptDistributionScale,conceptOffset,numSequentialConcepts,n).next() # some random observation...
y_pred=[]
for i in x[87500:]:
    y_pred.append(classifyOb(i, modelCollection))
import collections
count2=collections.Counter(y_pred )
count3=collections.Counter(y[87500:] )
print(count2,count3)
y_pred2=[]
for i in y_pred:
    if i==2:
        y_pred2.append(1)
    else:
        y_pred2.append(0)
count2=collections.Counter(y_pred2)
print(count2)

beginning stream


/root/anaconda3/envs/lr27/lib/python2.7/site-packages/ipykernel_launcher.py:72: ComplexWarning: Casting complex values to real discards the imaginary part


============  Done.  ============
It took 8.45229101181 seconds to train pcStream over a stream of 12500  63-dimensional observations.
   Number of clusters found: 5
   The clusters have the following number of assigned observations:  
       Cluster 0: 10000
       Cluster 1: 1399
       Cluster 2: 305
       Cluster 3: 147
       Cluster 4: 94
  
Predicting (scoring) new observation x:


In [163]:

import timeit

# If you use the source code or impliment pcStream, please cite the following paper:
# Mirsky, Yisroel, Bracha Shapira, Lior Rokach, and Yuval Elovici. "pcStream: A Stream Clustering Algorithm for Dynamically Detecting and Managing Temporal Contexts." In Advances in Knowledge Discovery and Data Mining (PAKDD), pp. 119-133. Springer International Publishing, 2015.

#### Demo ####

# generate a test dataset
conceptDuration = 1000
conceptDistributionScale = 1
conceptOffset = 3
numSequentialConcepts = 10
n = 63 #number of features

# DataGen = genDataset(conceptDuration,conceptDistributionScale,conceptOffset,numSequentialConcepts,n) #pcStream.genDataset is a python generator for testing purposes. It generates random sequential overalping normal distributions
# Dataset = list()

# for x in DataGen:
#     Dataset.append(x)
Dataset=x[:87500]
m = len(Dataset) #length of stream

# Run pcStream
driftThreshold = 8.7 # note, it is generally reccoemnded to zscore the dataset before hand. In which case, the driftThreshold is typically slightly above 1
maxDriftSize = int(n*1.5)
percentVarience = 0.60
modelMemory = 10000

pcS_args = (driftThreshold,maxDriftSize,percentVarience,modelMemory) #pcStream argumetns are passed to the function as a tuple
start = timeit.default_timer()	
Result =pcS(Dataset,pcS_args) #Result is the tuple (pcStreamModelCollection, clusterLabels)
stop = timeit.default_timer()
modelCollection  = Result[0]

print "============  Done.  ============"
print "It took " + str(stop-start) + " seconds to train pcStream over a stream of " + str(m) + "  " + str(n) + "-dimensional observations."
print "   Number of clusters found: " + str(len(modelCollection))
print "   The clusters have the following number of assigned observations:  "
for cluster in range(0,len(modelCollection)): #a pcStream cluster has the form (X,A,mu) -see the paper
    print "       Cluster " + str(cluster) + ": " + str(len(modelCollection[cluster][0]))

print "  "
print "Predicting (scoring) new observation x:"
#x =genDataset(conceptDuration,conceptDistributionScale,conceptOffset,numSequentialConcepts,n).next() # some random observation...



beginning stream


/root/anaconda3/envs/lr27/lib/python2.7/site-packages/ipykernel_launcher.py:72: ComplexWarning: Casting complex values to real discards the imaginary part


============  Done.  ============
It took 61.7211670876 seconds to train pcStream over a stream of 87500  63-dimensional observations.
   Number of clusters found: 357
   The clusters have the following number of assigned observations:  
       Cluster 0: 94
       Cluster 1: 349
       Cluster 2: 118
       Cluster 3: 192
       Cluster 4: 130
       Cluster 5: 115
       Cluster 6: 422
       Cluster 7: 216
       Cluster 8: 157
       Cluster 9: 94
       Cluster 10: 94
       Cluster 11: 143
       Cluster 12: 412
       Cluster 13: 141
       Cluster 14: 359
       Cluster 15: 94
       Cluster 16: 174
       Cluster 17: 276
       Cluster 18: 94
       Cluster 19: 127
       Cluster 20: 94
       Cluster 21: 94
       Cluster 22: 142
       Cluster 23: 222
       Cluster 24: 614
       Cluster 25: 171
       Cluster 26: 417
       Cluster 27: 94
       Cluster 28: 133
       Cluster 29: 166
       Cluster 30: 181
       Cluster 31: 94
       Cluster 32: 168
       Cluster 33: 151

In [164]:
y_pred=[]
for i in x[87500:]:
    y_pred.append(classifyOb(i, modelCollection))
import collections
count2=collections.Counter(y_pred )
count3=collections.Counter(y[87500:] )
print(count2,count3)

/root/anaconda3/envs/lr27/lib/python2.7/site-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


(Counter({170: 5178, 11: 4053, 315: 957, 355: 798, 356: 724, 335: 191, 57: 148, 279: 118, 56: 99, 7: 58, 12: 34, 109: 26, 174: 25, 6: 18, 349: 17, 17: 7, 41: 5, 47: 4, 48: 4, 84: 4, 351: 4, 98: 4, 187: 4, 312: 2, 314: 2, 242: 2, 1: 1, 223: 1, 16: 1, 288: 1, 296: 1, 311: 1, 334: 1, 345: 1, 207: 1, 225: 1, 103: 1, 104: 1, 111: 1, 340: 1}), Counter({0.0: 7379, 1.0: 5121}))


In [165]:
y_pred2=[]
for i in y_pred:
    if i==170:
        y_pred2.append(0)
    else:
        y_pred2.append(1)

In [166]:
count2=collections.Counter(y_pred2)
print(count2)

Counter({1: 7322, 0: 5178})


In [167]:

from sklearn.metrics import roc_curve, auc

import sklearn as sk
def evaluate(actual, predictions):
    FPR, TPR, thresholds = roc_curve(actual, predictions)
    cen = auc(FPR, TPR) 
    RightIndex=(TPR+(1-FPR)-1)
    index=np.argmax(RightIndex)
    tpr_val=TPR[index]
    fpr_val=FPR[index]
    thresholds_val=thresholds[index]
    y_pred=[0 if i<thresholds_val else 1 for i in predictions]
    y_pred_test=y_pred
    y_test_test=actual
    pre=sk.metrics.precision_score(y_test_test, y_pred_test)
    rec=sk.metrics.recall_score(y_test_test, y_pred_test)
    f1= sk.metrics.f1_score(y_test_test, y_pred_test)
    print("AUC", cen)
    print("Precision", pre)
    print( "Recall",rec)
    print( "f1_score", f1)
    mat=sk.metrics.confusion_matrix(y_test_test, y_pred_test)
    tp=mat[1][1]
    fn=mat[1][0]
    fp=mat[0][1]
    tn=mat[0][0]
    print(mat)
    print("TPR",tp*1.0/(tp+fn))
    print("FPR",fp*1.0/(tn+fp))
evaluate(y[87500:],y_pred2)

('AUC', 0.7410369161163644)
('Precision', 0.6087134662660475)
('Recall', 0.8703378246436243)
('f1_score', 0.7163867234589729)
[[4514 2865]
 [ 664 4457]]
('TPR', 0.8703378246436243)
('FPR', 0.38826399241089576)


In [162]:

from sklearn.metrics import roc_curve, auc

import sklearn as sk
def evaluate(actual, predictions):
    FPR, TPR, thresholds = roc_curve(actual, predictions)
    cen = auc(FPR, TPR) 
    RightIndex=(TPR+(1-FPR)-1)
    index=np.argmax(RightIndex)
    tpr_val=TPR[index]
    fpr_val=FPR[index]
    thresholds_val=thresholds[index]
    y_pred=[0 if i<thresholds_val else 1 for i in predictions]
    y_pred_test=y_pred
    y_test_test=actual
    pre=sk.metrics.precision_score(y_test_test, y_pred_test)
    rec=sk.metrics.recall_score(y_test_test, y_pred_test)
    f1= sk.metrics.f1_score(y_test_test, y_pred_test)
    print("AUC", cen)
    print("Precision", pre)
    print( "Recall",rec)
    print( "f1_score", f1)
    mat=sk.metrics.confusion_matrix(y_test_test, y_pred_test)
    tp=mat[1][1]
    fn=mat[1][0]
    fp=mat[0][1]
    tn=mat[0][0]
    print(mat)
    print("TPR",tp*1.0/(tp+fn))
    print("FPR",fp*1.0/(tn+fp))
evaluate(y[87500:],y_pred2)

('AUC', 0.9010169377418287)
('Precision', 0.9130164002491177)
('Recall', 0.8588166373755126)
('f1_score', 0.8850875427651439)
[[6960  419]
 [ 723 4398]]
('TPR', 0.8588166373755126)
('FPR', 0.056782761891855264)
